<a href="https://colab.research.google.com/github/patrixam/chatbot/blob/main/notebooks/proyecto_apoyo/Basado_Reglas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BASADO EN REGLAS (BIBLIOTECA EXPERTA)

In [ ]:
!pip install experta
!pip install frozendict==2.3.4

##Ejemplos base

In [ ]:
from experta import Fact, Rule, KnowledgeEngine

#Fact: Define hechos, que son las piezas de información con las que el sistema trabaja.
#Rule: Representa reglas que relacionan ciertos hechos con conclusiones o acciones.
#KnowledgeEngine: Es el motor que procesa las reglas y los hechos.

# Definir hechos
class Diagnostico(Fact):
    """
    Representa los síntomas del dispositivo.
    Los hechos contienen información específica sobre el estado del dispositivo.
    """
    pass

# Motor de conocimiento
class MotorDiagnostico(KnowledgeEngine):
    """
    Motor de reglas para diagnosticar problemas en un dispositivo
    basado en los síntomas declarados.
    """

    @Rule(Diagnostico(no_enciende=True, luces_apagadas=True))
    def cable_alimentacion(self):
        """
        Regla que se activa cuando el dispositivo no enciende y las luces están apagadas.
        """
        print("Diagnóstico: Revisar el cable de alimentación.")

# Crear una instancia del motor de diagnóstico
motor = MotorDiagnostico()

# Reiniciar el motor (limpia cualquier estado previo o hechos declarados)
motor.reset()

# Declarar hechos en el motor
# Estos hechos representan los síntomas observados en el dispositivo
motor.declare(Diagnostico(no_enciende=True, luces_apagadas=True))

# Ejecutar el motor para procesar las reglas basadas en los hechos declarados
motor.run()


Diagnóstico: Revisar el cable de alimentación.


In [ ]:
from experta import *

class Diagnostico(Fact):
    """Representa los síntomas reportados."""
    pass

class SistemaDomestico(KnowledgeEngine):

    # Regla 1: Problema de energía
    @Rule(Diagnostico(falla="no enciende", dispositivo="televisor"))
    def verificar_energia_televisor(self):
        print("Verifica si el televisor está enchufado y la toma de corriente funciona.")

    # Regla 2: Problema de pantalla negra
    @Rule(Diagnostico(falla="pantalla negra", dispositivo="ordenador"))
    def revisar_pantalla_ordenador(self):
        print("Prueba cambiar el cable HDMI o la pantalla del ordenador.")

    # Regla 3: Problema de conexión a internet
    @Rule(Diagnostico(falla="sin internet", dispositivo="router", indicador="luz roja"))
    def verificar_router(self):
        print("Reinicia el router y verifica la configuración de red.")

    # Regla 4: Problema de rendimiento
    @Rule(Diagnostico(falla="dispositivo lento", dispositivo="móvil"))
    def optimizar_moviles(self):
        print("Cierra aplicaciones en segundo plano y reinicia el dispositivo móvil.")

if __name__ == "__main__":
    print("=== Sistema de Diagnóstico Doméstico ===")

    # Crear instancia del motor
    engine = SistemaDomestico()
    engine.reset()

    # Declarar hechos
    engine.declare(Diagnostico(falla="no enciende", dispositivo="televisor"))
    engine.declare(Diagnostico(falla="pantalla negra", dispositivo="ordenador"))
    engine.declare(Diagnostico(falla="sin internet", dispositivo="router", indicador="luz roja"))

    # Ejecutar motor
    engine.run()
    print("=== Diagnóstico Completado ===")


=== Sistema de Diagnóstico Doméstico ===
Reinicia el router y verifica la configuración de red.
Prueba cambiar el cable HDMI o la pantalla del ordenador.
Verifica si el televisor está enchufado y la toma de corriente funciona.
=== Diagnóstico Completado ===


##Combinación con PLN

In [ ]:
from experta import KnowledgeEngine, Fact, Rule, MATCH
from transformers import pipeline

# ------------------------------------------
# 1) Definimos el hecho que contendrá
#    la pregunta, la etiqueta y la confianza.
# ------------------------------------------
class PreguntaUsuario(Fact):
    """
    Hecho (Fact) que representa la pregunta y su clasificación.
    Campos:
    - texto: contenido original de la pregunta
    - etiqueta: categoría/intención principal devuelta por el PLN
    - score: confianza asociada
    """
    texto = Field(str, mandatory=True)
    etiqueta = Field(str, default=None)
    score = Field(float, default=0.0)

# ------------------------------------------
# 2) Motor de Reglas con Experta
# ------------------------------------------
class MotorDeReglasFAQ(KnowledgeEngine):
    """
    Motor de Reglas que decide la respuesta en función de la etiqueta
    y la confianza devuelta por el modelo de clasificación.
    """

    @Rule(
        PreguntaUsuario(etiqueta="Horario", score=MATCH.score_pln),
        # Salience define prioridad (opcional)
        salience=10
    )
    def regla_horario(self, score_pln):
        # Ejemplo: si la confianza es mayor a 0.7, tomamos la respuesta de FAQ
        if score_pln >= 0.7:
            print("Regla disparada: Horario (FAQ).")
            self.declare(Fact(
                respuesta_final="Nuestro horario es de lunes a viernes, de 09:00 a 18:00."
            ))
            self.halted = True

    @Rule(
        PreguntaUsuario(etiqueta="Devoluciones", score=MATCH.score_pln),
        salience=10
    )
    def regla_devoluciones(self, score_pln):
        if score_pln >= 0.7:
            print("Regla disparada: Devoluciones (FAQ).")
            self.declare(Fact(
                respuesta_final="Aceptamos devoluciones hasta 30 días después de la compra."
            ))
            self.halted = True

    @Rule(
        PreguntaUsuario(etiqueta="Envíos", score=MATCH.score_pln),
        salience=10
    )
    def regla_envios(self, score_pln):
        if score_pln >= 0.7:
            print("Regla disparada: Envíos (FAQ).")
            self.declare(Fact(
                respuesta_final="Los envíos demoran entre 2 y 5 días laborables, según la ubicación."
            ))
            self.halted = True

    @Rule(
        PreguntaUsuario(etiqueta="Pagos", score=MATCH.score_pln),
        salience=10
    )
    def regla_pagos(self, score_pln):
        if score_pln >= 0.7:
            print("Regla disparada: Pagos (FAQ).")
            self.declare(Fact(
                respuesta_final="Aceptamos diversas formas de pago, incluyendo PayPal y tarjetas de crédito."
            ))
            self.halted = True

    # Regla fallback: si nada ha disparado respuesta_final, o la confianza < 0.7
    @Rule(
        PreguntaUsuario(score=MATCH.score_pln),
        NOT(Fact(respuesta_final=W())),  # No existe ningún hecho con 'respuesta_final'
        salience=-1                      # Baja prioridad
        )
    def regla_fallback(self, score_pln):
        """
        Si llegamos aquí, significa que no hay ninguna regla de FAQ que coincida
        o la confianza es baja. Devolvemos una respuesta genérica.
        """
        print("Regla fallback activada.")
        mensaje = (
            "Lo siento, no tengo una respuesta clara a tu pregunta. "
            "Por favor, contáctanos directamente para más información."
        )
        self.declare(Fact(respuesta_final=mensaje))
        self.halted = True

# ------------------------------------------
# 3) Función para clasificar y aplicar reglas
# ------------------------------------------
def procesar_pregunta(pregunta):
    """
    1) Usamos un pipeline de zero-shot classification en español para etiquetar la pregunta.
    2) Cargamos el motor de reglas y le pasamos los hechos (pregunta, etiqueta, score).
    3) Retornamos la respuesta final.
    """

    # Lista de etiquetas que manejamos en nuestro FAQ
    candidate_labels = ["Horario", "Devoluciones", "Envíos", "Pagos"]

    # Cargamos un modelo zero-shot multilingüe
    clasificador = pipeline(
        "zero-shot-classification",
        model="Recognai/bert-base-spanish-wwm-cased-xnli",
        tokenizer="Recognai/bert-base-spanish-wwm-cased-xnli"
    )

    # Clasificamos
    resultado = clasificador(
        sequences=pregunta,
        candidate_labels=candidate_labels,
        hypothesis_template="La pregunta está relacionada con {}."
        # Este template se usa para XNLI; ajusta según la doc.
    )
    # 'resultado' típicamente es un dict con:
    # {
    #   'labels': [...],
    #   'scores': [...],
    #   'sequence': ...
    # }
    # Donde labels[i] corresponde a la etiqueta candidate_labels[i] y scores[i] a la confianza.

    # Tomamos la etiqueta con mayor score
    etiqueta_principal = resultado["labels"][0]
    confianza = float(resultado["scores"][0])

    print(f"[PLN] Mejor etiqueta: {etiqueta_principal} (confianza={confianza:.2f})")

    # Ahora creamos el motor de reglas e inyectamos el hecho PreguntaUsuario
    motor = MotorDeReglasFAQ()
    motor.reset()

    motor.declare(
        PreguntaUsuario(
            texto=pregunta,
            etiqueta=etiqueta_principal,
            score=confianza
        )
    )

    motor.run()

    # Leemos la respuesta final de la base de hechos
    for fact_id, fact_data in motor.facts.items():
        if isinstance(fact_data, Fact) and "respuesta_final" in fact_data:
            return fact_data["respuesta_final"]

    # Por seguridad, en caso de que nada haya funcionado
    return "Lo siento, no tengo respuesta para tu pregunta."

# ------------------------------------------
# 4) Ejemplo de uso
# ------------------------------------------
if __name__ == "__main__":
    # - Ejemplo 1: Pregunta sobre envíos
    pregunta1 = "¿Cuánto tardan en llegar los productos?"
    resp1 = procesar_pregunta(pregunta1)
    print(f"\nPREGUNTA: {pregunta1}\nRESPUESTA: {resp1}")
    print("-"*50)

    # - Ejemplo 2: Pregunta sobre PayPal
    pregunta2 = "¿Aceptan pagos con PayPal?"
    resp2 = procesar_pregunta(pregunta2)
    print(f"\nPREGUNTA: {pregunta2}\nRESPUESTA: {resp2}")
    print("-"*50)

    # - Ejemplo 3: Pregunta no contemplada (más ambigua)
    pregunta3 = "¿Cómo puedo realizar un cambio de producto si me arrepiento?"
    resp3 = procesar_pregunta(pregunta3)
    print(f"\nPREGUNTA: {pregunta3}\nRESPUESTA: {resp3}")


Device set to use cpu


[PLN] Mejor etiqueta: Envíos (confianza=0.83)
Regla disparada: Envíos (FAQ).

PREGUNTA: ¿Cuánto tardan en llegar los productos?
RESPUESTA: Los envíos demoran entre 2 y 5 días laborables, según la ubicación.
--------------------------------------------------


Device set to use cpu


[PLN] Mejor etiqueta: Pagos (confianza=0.68)
Regla fallback activada.

PREGUNTA: ¿Aceptan pagos con PayPal?
RESPUESTA: Lo siento, no tengo una respuesta clara a tu pregunta. Por favor, contáctanos directamente para más información.
--------------------------------------------------


Device set to use cpu


[PLN] Mejor etiqueta: Envíos (confianza=0.51)
Regla fallback activada.

PREGUNTA: ¿Cómo puedo realizar un cambio de producto si me arrepiento?
RESPUESTA: Lo siento, no tengo una respuesta clara a tu pregunta. Por favor, contáctanos directamente para más información.
